# Iterative Reasoning Model - Kaggle Runner

Эта рабочая тетрадь демонстрирует работу с моделью итеративных рассуждений, включая обучение, оценку и визуализацию результатов.

In [ ]:
# Установка необходимых зависимостей
!pip install torch matplotlib seaborn scikit-learn

In [ ]:
# Импорт необходимых библиотек
import os
import torch
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Импорт модулей проекта
from model import IterativeReasoningModel, ReasoningDataset
from train_model import train_model
from evaluate_model import evaluate_model
from viz_tools import plot_attention_heatmap, plot_representation_trajectory

# Определение пути для данных и выходных файлов
data_dir = Path('/kaggle/input/reasoning-dataset')
output_dir = Path('/kaggle/working')

# Создание директорий, если они не существуют
output_dir.mkdir(exist_ok=True)

# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")

## Загрузка и подготовка данных

Загрузим словарь и наборы данных для обучения, валидации и тестирования.

In [ ]:
# Пути к файлам данных
vocab_path = data_dir / "vocab.json"
train_path = data_dir / "train.json"
valid_path = data_dir / "valid.json"
test_path = data_dir / "test.json"

# Проверка существования файлов данных
if not all(p.exists() for p in [vocab_path, train_path, valid_path, test_path]):
    print("Файлы данных не найдены, копирование их из репозитория...")
    
    # Копирование файлов из репозитория
    import shutil
    data_dir.mkdir(exist_ok=True)
    
    repo_data_dir = Path('data')
    for file_name in ['vocab.json', 'train.json', 'valid.json', 'test.json']:
        source = repo_data_dir / file_name
        target = data_dir / file_name
        if source.exists():
            shutil.copy(source, target)
            print(f"Скопирован файл {file_name}")

# Загрузка наборов данных
train_dataset = ReasoningDataset(str(train_path), str(vocab_path))
val_dataset = ReasoningDataset(str(valid_path), str(vocab_path))
test_dataset = ReasoningDataset(str(test_path), str(vocab_path))

print(f"Размер словаря: {len(train_dataset.vocab)}")
print(f"Примеров для обучения: {len(train_dataset)}")
print(f"Примеров для валидации: {len(val_dataset)}")
print(f"Примеров для тестирования: {len(test_dataset)}")

## Создание и обучение модели

Создадим модель итеративного рассуждения и обучим ее на тренировочных данных.

In [ ]:
# Параметры модели
d_model = 512
num_heads = 8
d_ff = 2048
num_layers = 8
max_iterations = 20
dropout = 0.1

# Параметры обучения
batch_size = 32
num_epochs = 500  # Можно увеличить для достижения гроккинга
learning_rate = 1e-5
weight_decay = 0.01
early_stopping_patience = 20

# Создание модели
model = IterativeReasoningModel(
    vocab_size=len(train_dataset.vocab),
    d_model=d_model,
    num_heads=num_heads,
    d_ff=d_ff,
    num_layers=num_layers,
    max_iterations=max_iterations,
    dropout=dropout
)

# Вывод количества параметров модели
num_params = sum(p.numel() for p in model.parameters())
print(f"Количество параметров модели: {num_params:,}")

# Создание директории для контрольных точек
checkpoint_dir = output_dir / "checkpoints"
checkpoint_dir.mkdir(exist_ok=True)

# Обучение модели
model = train_model(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    batch_size=batch_size,
    num_epochs=num_epochs,
    learning_rate=learning_rate,
    device=str(device),
    weight_decay=weight_decay,
    early_stopping_patience=early_stopping_patience,
    checkpoint_dir=str(checkpoint_dir),
    use_mixed_precision=True,
    use_consistency_loss=True,
    consistency_weight=0.1,
    gradient_accumulation_steps=1
)

# Сохранение обученной модели
model_path = output_dir / "model.pt"
torch.save(model.state_dict(), str(model_path))
print(f"Модель сохранена в {model_path}")

## Оценка модели

Оценим производительность модели на тестовом наборе данных.

In [ ]:
# Оценка модели
metrics = evaluate_model(
    model=model,
    test_dataset=test_dataset,
    batch_size=batch_size,
    device=str(device)
)

# Сохранение метрик в файл
metrics_path = output_dir / "metrics.json"
with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=2)

# Визуализация метрик
type_names = list(metrics.keys())
accuracies = [metrics[t]['correct'] / metrics[t]['total'] for t in type_names]
counts = [metrics[t]['total'] for t in type_names]

plt.figure(figsize=(12, 6))
bars = plt.bar(type_names, accuracies, color='skyblue')
plt.title('Точность по типам данных')
plt.xlabel('Тип')
plt.ylabel('Точность')
plt.xticks(rotation=45, ha='right')

# Добавление текста с точностью над каждым столбцом
for bar, acc, count in zip(bars, accuracies, counts):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
             f"{acc:.2f}\n({count})", ha='center')

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(output_dir / "accuracy_by_type.png")
plt.show()

## Визуализация внимания и процесса рассуждения

Визуализируем механизм внимания и процесс итеративного рассуждения для нескольких примеров.

In [ ]:
# Создание словаря, отображающего ID токена в читаемое представление
token_ids_to_words = {idx: token for idx, token in enumerate(train_dataset.vocab)}

# Выбор нескольких примеров для визуализации
viz_examples = [test_dataset[i] for i in range(3)]

for i, example in enumerate(viz_examples):
    print(f"\nПример {i+1}:")
    
    # Преобразование входных токенов в читаемый текст
    input_text = ' '.join([token_ids_to_words.get(t.item(), f"<{t.item()}>") for t in example['input_ids']])
    target_text = ' '.join([token_ids_to_words.get(t.item(), f"<{t.item()}>") for t in example['target_ids']])
    
    print(f"Входной текст: {input_text}")
    print(f"Целевой текст: {target_text}")
    print(f"Тип: {example['type']}")
    
    # Подготовка входных данных
    input_ids = example['input_ids'].unsqueeze(0).to(device)
    
    # Визуализация внимания
    attention_path = output_dir / f"attention_example_{i+1}.png"
    model.visualize_attention(
        tokens=input_ids,
        token_ids_to_words=token_ids_to_words,
        mask_token_id=test_dataset.mask_token_id,
        save_path=str(attention_path)
    )
    
    # Визуализация процесса рассуждения
    reasoning_path = output_dir / f"reasoning_example_{i+1}.png"
    model.visualize_iterative_reasoning(
        tokens=input_ids,
        mask_token_id=test_dataset.mask_token_id,
        token_ids_to_words=token_ids_to_words,
        save_path=str(reasoning_path)
    )
    
    # Показ результатов
    plt.figure(figsize=(16, 8))
    plt.subplot(1, 2, 1)
    plt.imshow(plt.imread(attention_path))
    plt.axis('off')
    plt.title("Карта внимания")
    
    plt.subplot(1, 2, 2)
    plt.imshow(plt.imread(reasoning_path))
    plt.axis('off')
    plt.title("Траектория рассуждения")
    
    plt.tight_layout()
    plt.savefig(output_dir / f"visualization_example_{i+1}.png", dpi=300)
    plt.show()

## Тестирование влияния количества итераций на точность

Проверим, как изменяется точность модели при разном количестве итераций рассуждения.

In [ ]:
# Список количеств итераций для тестирования
iterations_list = [1, 2, 5, 10, 15, 20, 25, 30]

# Создаем подмножество тестовых данных для быстрого тестирования
subset_indices = torch.randperm(len(test_dataset))[:100]
subset_dataset = torch.utils.data.Subset(test_dataset, subset_indices)

# Создаем даталоадер
subset_loader = torch.utils.data.DataLoader(
    subset_dataset, batch_size=batch_size, shuffle=False
)

# Список для хранения результатов
accuracies = []

# Тестирование с разным числом итераций
model.eval()
for num_iterations in iterations_list:
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(subset_loader):
            input_ids = batch[0]['input_ids'].to(device)
            target_ids = batch[0]['target_ids'].to(device)
            
            # Генерация предсказаний с заданным числом итераций
            predictions = model.generate(
                input_ids,
                mask_token_id=test_dataset.mask_token_id,
                num_iterations=num_iterations
            )
            
            # Подсчет точности
            matches = (predictions == target_ids).all(dim=1)
            correct += matches.sum().item()
            total += len(matches)
    
    accuracy = correct / total if total > 0 else 0
    accuracies.append(accuracy)
    print(f"Количество итераций: {num_iterations}, Точность: {accuracy:.4f} ({correct}/{total})")

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.plot(iterations_list, accuracies, marker='o', linestyle='-', linewidth=2, markersize=8)
plt.title('Влияние количества итераций на точность')
plt.xlabel('Количество итераций')
plt.ylabel('Точность')
plt.grid(True, linestyle='--', alpha=0.7)

# Добавление текста с точностью
for i, (x, y) in enumerate(zip(iterations_list, accuracies)):
    plt.text(x, y, f"{y:.4f}", ha='center', va='bottom')

plt.tight_layout()
plt.savefig(output_dir / "iterations_vs_accuracy.png", dpi=300)
plt.show()

## Заключение

В этом ноутбуке мы обучили, оценили и визуализировали работу модели итеративного рассуждения. Результаты показывают эффективность подхода итеративного рассуждения, особенно для задач, требующих логического анализа и интеграции разрозненной информации.